# EO Exploitation Platform Common Architecture (EOEPCA) Overview

EOEPCA provides a <span style="color:blue">**blueprint architecture**</span> and <span style="color:blue">**open-source reference implementation**</span> for an Exploitation Platform.

An <span style="color:blue">**exploitation platform**</span> provides an online virtual workspace with access to large volumes of data, and cloud-hosted tooling for <span style="color:blue">**analysis and processing close-to-the-data**</span>.

The key features of an exploitation platform include:
* Data Search & Discovery
* Data Access
* User-defined Processing & Analysis

The architecture is defined as a set <span style="color:blue">**building blocks**</span> that expose their services through <span style="color:blue">**open standard interfaces**</span> - an approach that is designed to encourage <span style="color:blue">**federation and interroperation**</span> amongst platforms.

The <span style="color:darkorange">**Resource Catalogue**</span> and <span style="color:darkorange">**Data Access**</span> building blocks provide core resource management capabilities, and the <span style="color:darkorange">**ADES**</span> provides hosted user-defined processing capabilities...

<img src="../images/reference-impl.png" alt="Reference Implementation" style="display:block;margin-left:auto;margin-right:auto;width:70%;"/>

# Demonstration of Resource Catalogue & Data Access services

We demonstrate the **Resource Catalogue** and **Data Access** building blocks - which allow to ingest, search, discover, visualise and access data.<br>
Primarily these are <span style="color:blue">**back-end services**</span> that provide <span style="color:blue">**standards-based API interfaces**</span> upon which platforms can be developed. Nevertheless they do provide their own user interfaces.

<span style="color:steelblue">**Use of open standard API interfaces allows seamless use of commonly-used clients, such as QGIS and typical python libraries (owslib, pystac_client, ...)**</span>

## Demo Starting Point

* <span style="color:red">**Kubernetes**</span> cluster<br>
  Running in minikube, on a VM hosted in CREODIAS<br>
  Provides <span style="color:blue">**scalability and resilience to failure**</span>
* <span style="color:red">**Services**</span> running:
  * Resource Catalogue (OGC CSW, OpenSearch, STAC, API Records)
  * Data Access (OGC WMS, WCS, OpenSearch)
  * Minio (S3 object storage)
* Four <span style="color:red">**collections**</span> in catalogue:
  * S2MSI1C (Sentinel-2 MultiSpectral Instrument Level 1C)
  * S2MSI2A (Sentinel-2 MultiSpectral Instrument Level 2A)
  * L8MSI1TP (Landsat-8 Level 1TP)
  * L8MSI1GT (Landsat-8 Level 1GT)
* <span style="color:red">**Data harvested**</span> from CREODIAS over Central Europe
  * Sentinel-2 for 30 Aug 2022
  * Landsat-8 for August 2022

In [ ]:
base_domain = "demo.guide.eoepca.org"
from IPython.display import display, Markdown, Javascript
import warnings
warnings.simplefilter("ignore")

In [ ]:
%%html
<style>table {float:left}</style>

## Resource Catalogue Web Client

The Resource Catalogue provides a standards-compliant metadata catalogue for search and discovery.<br>
It is based on [pycsw](https://pycsw.org/) with a [PostGIS](https://postgis.net/) database.

<img src="../images/resource-catalogue.png" alt="Resource Catalogue" style="display:block;margin-left:auto;margin-right:auto;width:30%;"/>

### Catalogue Web UI

The primary utility of the catalogue are its service interfaces for data discovery and access to metadata. Nevertheless it does provide a simple web interface to browse the metadata holding.

#### Open the Web UI

In [ ]:
catalogue_root = f"https://resource-catalogue-open.{base_domain}"
print(f"Catalogue URL: {catalogue_root}")
display(Javascript(f"window.open('{catalogue_root}')"))
display(Markdown(f'''
### Inspect pre-loaded data
Observe the existing data for 30th August 2022 in the collections for Sentinel-2 and Landsat-8...
* **Full `S2MSI2A` collection metadata**: {catalogue_root}/collections/S2MSI2A/items?f=json
* **Specific product metadata**: {catalogue_root}/collections/S2MSI2A/items/S2A_MSIL2A_20220830T094601_N0400_R079_T33UXQ_20220830T142602.SAFE?f=json
'''))
display(Markdown(f'''
### Inspect the landing pages for the various service interfaces

| Page | Path |
| ------- | ---- |
| **Conformance** | [`/conformance`]({catalogue_root}/conformance?f=json) |
| **CSW 3.0.0 Capabilties** | [`/csw`]({catalogue_root}/csw) |
| **CSW 2.0.2 Capabilties** | [`/csw?service=CSW&version=2.0.2&request=GetCapabilities`]({catalogue_root}/csw?service=CSW&version=2.0.2&request=GetCapabilities) |
| **OpenSearch Description Document** | [`/opensearch`]({catalogue_root}/opensearch) |
| **OGC API Records** | [`/`]({catalogue_root}/?f=json) |
| **STAC API** | [`/search`]({catalogue_root}/search) |
'''))

## Data Access View Server Web Client

The Data Access service includes services for data discovery (OpenSearch), visualisation (WMS/WMTS) and access/download (WCS).

<img src="../images/data-access.png" alt="Data Access" style="display:block;margin-left:auto;margin-right:auto;width:40%;"/>

### View Server Web Client

The View Server provides a web UI for discovery, visualisation and download of data.

In [ ]:
data_root = f"https://data-access-open.{base_domain}"
print(f"View Server URL: {data_root}")
display(Javascript(f"window.open('{data_root}/?x=16.949954&y=47.908475&start=2022-08-01T00%3A00%3A00Z&end=2022-09-01T00%3A00%3A00Z&z=8&S2L1C_search=false&S2L1C_visible=false&S2L2A_search=false&S2L2A_visible=false')"))

#### **Summary of UI Capabilities**

* <span style="color:blue">**Map View:**</span> Pan and Zoom to area of interest
* <span style="color:blue">**Timeslider:**</span> Summarises temporal data distribution and provides temporal navigation and selection
  * Select data item to zoom map
* <span style="color:blue">**Search Results:**</span> Within current area and time of interest
  * Inspect product details by hitting (i)
  * Select to add to basket
* <span style="color:blue">**Basket:**</span> Download data
* <span style="color:blue">**Filters:**</span> Refine the search
  * Time and spatial filters are already applied via the map and timeslider
  * Draw to refine spatial filters
* <span style="color:blue">**Layers:**</span> Products, Base-layers and Overlays

#### **Configurable Layers**

Layers are configured for each Product Type in the service configuration - mapping the bands into the visible colours, with support for expressions...
```
      browses:
        TRUE_COLOR:
          asset: visual
          red:
            expression: B04
            range: [0, 4000]
            nodata: 0
          green:
            expression: B03
            range: [0, 4000]
            nodata: 0
          blue:
            expression: B02
            range: [0, 4000]
            nodata: 0
        FALSE_COLOR:
          red:
            expression: B08
            range: [0, 4000]
            nodata: 0
          green:
            expression: B04
            range: [0, 4000]
            nodata: 0
          blue:
            expression: B03
            range: [0, 4000]
            nodata: 0
        NDVI:
          grey:
            expression: (B08-B04)/(B08+B04)
            range: [-1, 1]
```

#### **Kubernetes - Scalability and Resilience**

Kubernetes provides scalability and resilience:
* Multiple nodes for horizontal and vertical system scaling
* Replicas for scaling of stateless services
* Automatic restart of failed processes - with probes for preemptive health monitoring

**_Illustratration of multiple stateless replicas of the 'renderer' service..._**

In [ ]:
!kubectl -n rm get pod

## Data Access Cache

The data access service maintains a cache in S3 object storage. Data can be pre-seeded into the cache during harvesting.

In [ ]:
minio_root = f"http://minio-console.{base_domain}"
print(f"Minio object storage URL: {minio_root}")
display(Javascript(f"window.open('{minio_root}')"))

## Harvest Data Over UK

Our starting point has harvested data in central europe.

To illustrate harvesting we can harvest some Landsat-8 data - over the UK for August 2022.

The harvester is tasked with a file that describes the job, in particular:
* URL and type (e.g. OpenSearch) of the source
* Selection filter - typically spatial and temporal

### Harvester configuration

This illustrates harvesting from an **OpenSearch** endpoint.

Other types supported include **STAC API** and **Static STAC catalogue (STAC files)**.

In [ ]:
!cat ${HOME}/deployment-guide/deploy/samples/harvester/config-Landsat8-2022.08_UK.yaml

### Invoke the Harvester with the configuration

In [ ]:
!${HOME}/deployment-guide/deploy/bin/harvest ${HOME}/deployment-guide/deploy/samples/harvester/config-Landsat8-2022.08_UK.yaml

### Check New Data in Services

#### Resource Catalogue

In [ ]:
display(Javascript(f"window.open('{catalogue_root}/collections/L8MSI1TP/items')"))

#### Data Access View Server

In [ ]:
display(Javascript(f"window.open('{data_root}/?x=0.22&y=51.66&z=7&start=2022-08-01T00%3A00%3A00Z&end=2022-09-01T00%3A00%3A00Z&z=6&S2L1C_search=false&S2L1C_visible=false&S2L2A_search=false&S2L2A_visible=false')"))

# Client Visualisation

The Data Access provides standard OGC WMS interfaces that can be exploited by any client supporting this standard.

Here we present two such examples...
* [**Leaflet**](https://leafletjs.com/) javascript client in the Jupyter notebook
* [**QGIS**](https://www.qgis.org/) application for GIS

## Leaflet WMS Client

Here we initialise Leaflet with the **Landsat-8 `L8L1TP__TRUE_COLOR` layer**.

In [ ]:
from owslib.wms import WebMapService
wms_endpoint = f"{data_root}/ows"
wms = WebMapService(wms_endpoint, version='1.3.0')
layer_id = "L8L1TP__TRUE_COLOR"
layer_bbox = wms[layer_id].boundingBoxWGS84
centreLong = layer_bbox[0] + (layer_bbox[2]-layer_bbox[0])/2
centreLat = layer_bbox[1] + (layer_bbox[3]-layer_bbox[1])/2
centre = [centreLat, centreLong]  # lat, long

print(f"Layer...\n  ID: {layer_id}\n  bbox: {layer_bbox}\n  centre (long/lat): [{centreLong} / {centreLat}]\n  WMS: {wms_endpoint}")
print("Layers...")
list(wms.contents)

In [ ]:
from ipyleaflet import Map, WMSLayer, basemaps
from ipywidgets import Layout

wms = WMSLayer(
    url=wms_endpoint,
    layers='L8L1TP__TRUE_COLOR',
    format='image/png',
    transparent=True
)

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(centreLat, centreLong), zoom=6, layout=Layout(width='90%', height='800px'))
m.add_layer(wms)
m

## Demonstration with QGIS
### Discover Data with MetaDearch Plugin

In [ ]:
display(Markdown(f'''
Using the MetaSearch tool in QGIS we can connect to the Catalogue CSW endpoint - {catalogue_root}/csw
* Add service {catalogue_root}/csw
* Get Service Info (GetCapabilities)
* Search for records
* Select record to obtain detailed metadata
* `Add Data` to add as a WMS layer to the map
'''))

### Visualise Layer via WMS

In [ ]:
display(Markdown(f'''
Add `WMS/WMTS` service with the URL - {data_root}/ows?service=WMS&version=1.3.0&request=GetCapabilities<br>
Select a layer to add to map - e.g. `L8L1TP__TRUE_COLOUR`
'''))

## Demonstration of Service Endpoints

Demonstration of Resource Catalogue and Data Access interfaces...
* Resource Catalogue: OGC CSW, OpenSearch, STAC, API Records
* Data Access: WMS, WCS

Uses, amongst others, [OWSLib](https://geopython.github.io/OWSLib) - a Python package for client programming with Open Geospatial Consortium (OGC) web service interface standards, and their related content models.

### Discovery - CSW
Data discovery using OGC CSW.

In [ ]:
csw_endpoint = f'{catalogue_root}/csw'
from owslib.csw import CatalogueServiceWeb
csw = CatalogueServiceWeb(csw_endpoint, timeout=30)
print(f"Service: {csw.identification.type} version {csw.version}")
print(f"Operations: {[op.name for op in csw.operations]}")

#### Constraints

In [ ]:
for constraint in csw.get_operation_by_name('GetRecords').constraints:
    print(f"{constraint.name}...\n{constraint.values}\n")

#### Discovery - Filter

Spatial query with bounding box...

In [ ]:
from owslib.fes import BBox
bbox_query = BBox([46, 15, 50, 18])
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

Or more complex with spatial (`bbox`), temporal (`time`), and attribute (`apiso:CloudCover`) filters combined with logical operators like and/or etc...

In [ ]:
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, SortBy, SortProperty
filter_list = [
    And(
        [
            bbox_query,
            PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2022-08-01 00:00'),
            PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2022-09-01 00:00'),
            PropertyIsLessThanOrEqualTo(propertyname='apiso:CloudCover', literal='20')
        ]
    )
]
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification.identtype}\ntitle: {csw.records[rec].identification.title}\n')

#### Discovery - By collection
Another option is to perform a collection level search, using the apiso:parentIdentifier queryable. Here only the Sentinel2 L1C datasets will be discovered.

In [ ]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'S2MSI1C')
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

#### Discovery - Free Text

In [ ]:
anytext_query = PropertyIsEqualTo('csw:AnyText', 'Orthoimagery')
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            anytext_query # any text
        ]
    )
]
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

#### Discovery - Record Details

In [ ]:
record_id = str(list(csw.records)[0])
csw.getrecordbyid(id=[record_id])
detailed_record = csw.records[record_id]

print(f"Title: {detailed_record.title}")
print("References...")
for ref in detailed_record.references:
    print(ref)

Bounding box...

In [ ]:
print("Bounding box: (%s, %s, %s, %s)" % (detailed_record.bbox.miny, detailed_record.bbox.minx, detailed_record.bbox.maxy, detailed_record.bbox.maxx))

Visualise spatial extent on map...

In [ ]:
import folium
m = folium.Map(location=[detailed_record.bbox.miny, detailed_record.bbox.minx], zoom_start=8, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(detailed_record.bbox.miny), float(detailed_record.bbox.minx)], [float(detailed_record.bbox.maxy), float(detailed_record.bbox.maxx)]]).add_to(m)
m

### OpenSearch Interface

In [ ]:
opensearch_endpoint = f'{catalogue_root}/csw?service=CSW&version=3.0.0&mode=opensearch&request=GetCapabilities'

#### OpenSearch - discovery template (using a simple http GET request)...

In [ ]:
import requests
from bs4 import BeautifulSoup
S = requests.Session()
R = S.get(url=opensearch_endpoint)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

#### OpenSearch - get individual record

In [ ]:
url = f'{opensearch_endpoint}&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&recordids={record_id}'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

### STAC API
Using `pystac_client` python client.

In [ ]:
from pystac_client import Client
stac = Client.open(catalogue_root)
print(f"URL: {catalogue_root}")
print(f"STAC service...\n  ID: {stac.id}\n  Title: {stac.title}\n  Description: {stac.description}")

#### Search using bounding box...

In [ ]:
mysearch = stac.search(collections=['metadata:main'], bbox=[15,46,18,50], max_items=50)
print(f"{mysearch.matched()} items found")

#### List record IDs of results...

In [ ]:
items = mysearch.get_items()
for item in items:
    print(item.id)

### OGC API Records
Using `owslib` client for OGC API Records.

In [ ]:
from owslib.ogcapi.records import Records
ogc_records = Records(catalogue_root)
print(f"URL: {ogc_records.url}")
ogc_records.conformance()

#### List Collections

In [ ]:
collections = ogc_records.collections()["collections"]
print(f"Number of collections: {len(collections)}")
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(collections)

#### Work with selected collection

In [ ]:
collection_name = "S2MSI2A"
my_query = ogc_records.collection_items(collection_name)
print(f"Number of records in collection {collection_name} = {my_query['numberMatched']}\n")
print("Summary of first record...")
my_query['features'][0]['properties']

#### Query with Bounding Box

In [ ]:
my_query = ogc_records.collection_items(collection_name, bbox=[15.34, 46.86, 15.44, 46.96])
print(f"Number of results = {my_query['numberMatched']}\n")
print("Summary of first record...")
my_query['features'][0]['properties']

#### Query using CQL

In [ ]:
my_query = ogc_records.collection_items(collection_name, filter="title LIKE 'S2A_MSIL2A_%TXN%'")
# my_query = ogc_records.collection_items(collection_name)
print(f"Number of results = {my_query['numberMatched']}\n")
print("Summary of first record...")
my_query['features'][0]['properties']